# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('hacks').master('local').getOrCreate()

In [30]:
data = spark.read.csv('hack_data.csv', inferSchema=True, header=True)
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [31]:
data.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [32]:
data.show(3)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
only showing top 3 rows



In [33]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(
    inputCols=[
        'Session_Connection_Time',
        'Bytes Transferred',
        'Kali_Trace_Used',
        'Servers_Corrupted',
        'Pages_Corrupted',
        'WPM_Typing_Speed'],
    outputCol='features'
)

final_data = assembler.transform(data)

In [34]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

scalerModel = scaler.fit(final_data)

In [35]:
final_data = scalerModel.transform(final_data)
final_data.show(3)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|      scaledFeatures|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|[0.56785108466505...|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|[1.41962771166263...|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58

In [36]:
from pyspark.ml.clustering import KMeans

km2 = KMeans(featuresCol='scaledFeatures', k=2)
km3 = KMeans(featuresCol='scaledFeatures', k=3)

km2_model = km2.fit(final_data)
km3_model = km3.fit(final_data)

In [37]:
print('H1: 2 Hackers')
print('\n- Within Set Sum of Squared Error = '+str(km2_model.computeCost(final_data)))
print('\n- Cluster Centers :')
for center in km2_model.clusterCenters():
    print(center)
print('\n- Clustered Sessions:')
km2_model.transform(final_data).select('prediction').groupby('prediction').count().show()

H1: 2 Hackers

- Within Set Sum of Squared Error = 601.7707512676716

- Cluster Centers :
[1.26023837 1.31829808 0.99280765 1.36491885 2.5625043  5.26676612]
[2.99991988 2.92319035 1.05261534 3.20390443 4.51321315 3.28474   ]

- Clustered Sessions:
+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



In [38]:
print('H0: 3 Hackers')
print('\n- Within Set Sum of Squared Error = '+str(km3_model.computeCost(final_data)))
print('\n- Cluster Centers :')
for center in km3_model.clusterCenters():
    print(center)
print('\n- Clustered Sessions:')
km3_model.transform(final_data).select('prediction').groupby('prediction').count().show()

H0: 3 Hackers

- Within Set Sum of Squared Error = 434.1492898715845

- Cluster Centers :
[1.21780112 1.37901802 1.99757683 1.37198977 2.55237797 5.29152222]
[2.99991988 2.92319035 1.05261534 3.20390443 4.51321315 3.28474   ]
[1.30217042 1.25830099 0.         1.35793211 2.57251009 5.24230473]

- Clustered Sessions:
+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   84|
|         0|   83|
+----------+-----+



In [39]:
# k=2일 때 더 뚜렷하게 분리됨

In [42]:
for k in range(2,9):
    kmeans = KMeans(featuresCol='scaledFeatures', k=k)
    model = kmeans.fit(final_data)
    wssse = model.computeCost(final_data)
    print('With k = {}, wssse = {}'.format(k,wssse))

With k = 2, wssse = 601.7707512676716
With k = 3, wssse = 434.1492898715845
With k = 4, wssse = 416.59452021895555
With k = 5, wssse = 250.8996394957951
With k = 6, wssse = 227.1906816083445
With k = 7, wssse = 213.51526316824572
With k = 8, wssse = 201.1116804267026
